In [12]:
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [13]:
gmsh.open("twist2D-2.geo")

Info    : Reading 'twist2D-2.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 30%] Meshing curve 3 (Line)
Info    : [ 50%] Meshing curve 4 (Line)
Info    : [ 60%] Meshing curve 5 (Line)
Info    : [ 80%] Meshing curve 6 (Line)
Info    : [ 90%] Meshing curve 7 (Line)
Info    : Done meshing 1D (Wall 0.000463471s, CPU 0.00045s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 50%] Meshing surface 2 (Transfinite)
Info    : Done meshing 2D (Wall 6.4732e-05s, CPU 6.2e-05s)
Info    : 15 nodes 36 elements
Info    : Done reading 'twist2D-2.geo'


In [14]:
mat1 = FEM.material("body1", E=2.e5, ν=0.3)
mat2 = FEM.material("body2", E=2.e5, ν=0.3)
problem = FEM.Problem([mat1, mat2], type="AxiSymmetric", bandwidth="none")

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 3)


LowLevelFEM.Problem("twist2D-2", "AxiSymmetric", 2, [("body1", 200000.0, 0.3, 7.85e-9), ("body2", 200000.0, 0.3, 7.85e-9)], 1.0, 15)

In [4]:
#gmsh.fltk.run()

In [5]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [15]:
K = FEM.stiffnessMatrix(problem)

30×30 SparseArrays.SparseMatrixCSC{Float64, Int64} with 300 stored entries:
⎡⠛⣤⠀⠀⠀⠀⣿⣤⠀⠀⠀⠛⠀⣤⠀⎤
⎢⠀⠀⠛⣤⠀⠀⠀⠀⠛⣿⣤⠀⠀⠀⣤⎥
⎢⠀⠀⠀⠀⠛⣤⠀⣤⣤⠀⠛⠛⣿⠛⣤⎥
⎢⠛⣿⠀⠀⠀⣤⠛⣤⠀⠀⠀⠛⣤⣿⠀⎥
⎢⠀⠀⣿⣤⠀⠛⠀⠀⣿⣿⠀⠀⠀⠀⣿⎥
⎢⣤⠀⠀⠛⣿⠀⣤⠀⠀⠀⠛⣤⠛⣤⠛⎥
⎢⠀⣤⠀⠀⣿⠛⣤⣿⠀⠀⠛⣤⣿⣿⠛⎥
⎣⠀⠀⠀⠛⠀⠛⠀⠀⠛⠛⠛⠀⠛⠀⠛⎦

In [16]:
supp = FEM.displacementConstraint("supp", uy=0)
supp2 = FEM.displacementConstraint("load", uy=0.0001)
load = FEM.load("load", fy=1)

("load", 0, 1, 0)

In [17]:
f = FEM.loadVector(problem, [load]) * 0

30-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [18]:
FEM.applyBoundaryConditions!(problem, K, f, [supp, supp2])

In [19]:
q = FEM.solveDisplacement(K, f)

30-element Vector{Float64}:
 -1.5000000000000078e-5
  0.0
 -3.000000000000009e-5
  0.0
 -2.9999999999999997e-5
  0.0001
 -1.5000000000000009e-5
  0.0001
 -1.5000000000000072e-5
  5.000000000000005e-5
  ⋮
  7.500000000000002e-5
 -1.5000000000000075e-5
  2.5000000000000015e-5
 -2.2500000000000066e-5
  5.000000000000003e-5
 -2.2500000000000083e-5
  2.499999999998101e-5
 -2.2500000000000042e-5
  7.500000000000002e-5

In [ ]:
S = FEM.solveStress(problem, q)

In [22]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)

In [20]:
u = FEM.showDoFResults(problem, q, "uvec", name="uvec", visible=false)
ux = FEM.showDoFResults(problem, q, "ux", name="ux", visible=false)
uy = FEM.showDoFResults(problem, q, "uy", name="uy", visible=false)
uz = FEM.showDoFResults(problem, q, "uz", name="uz", visible=false)
s = FEM.showStressResults(problem, S, "s", name="σ", visible=true, smooth=true)
sx = FEM.showStressResults(problem, S, "sx", name="σx", visible=false, smooth=true)
sy = FEM.showStressResults(problem, S, "sy", name="σy", visible=false, smooth=true)
sz = FEM.showStressResults(problem, S, "sz", name="σz", visible=false, smooth=true)
sxy = FEM.showStressResults(problem, S, "sxy", name="τxy", visible=false, smooth=true)
syz = FEM.showStressResults(problem, S, "syz", name="τyz", visible=false, smooth=true)
szx = FEM.showStressResults(problem, S, "szx", name="τzx", visible=false, smooth=true)

UndefVarError: UndefVarError: `S` not defined

In [14]:
FEM.plotOnPath(problem, "path", sx, 100, name="σx", visible=false);
FEM.plotOnPath(problem, "path", sxy, 100, name="τxy", visible=false);
FEM.plotOnPath(problem, "path", ux, 100, name="ux", visible=false);

In [15]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [21]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.2
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.6.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [11]:
gmsh.finalize()